## Import and initialize

In [1]:
import qcodes as qc
from qcodes import ManualParameter, ScaledParameter
from qcodes import Parameter, Instrument, validators as vals
from qcodes.instrument.channel import InstrumentChannel

pyqtgraph plotting not supported, try "from qcodes.plots.pyqtgraph import QtPlot" to see the full error


In [2]:
from qcodes.tests.instrument_mocks import DummyInstrument

In [3]:
dac = DummyInstrument('dac', gates=['dac1', 'dac2', 'dac3'])
dmm = DummyInstrument('dmm', gates=['voltage'])
lockin = DummyInstrument('lockin', gates=['X', "SineOut"])

In [4]:
print(dac.dac1.unit)

V


In [19]:
print('R = ' + str(12906/g()/1e3) + ' kOhms')

R = 9.500000000000002 kOhms


This is a typical value when checking a 'sample simulator' set to 10k with the IVVI rack

## Class definitions
Save all info about chip, device type, lines, series resistance, etc. as parameters in various classes so that it all gets stored in the snapshot

In [23]:
#Dictionaries below are for testing purposes.  Eventually will probably import these from config 
#files associated with each fridge and PCB

#Fridge series resistance dictionary
Rs_fridge = {'1-1': 3.4e3,
            '1-2': 3.4e3,
            '1-3': 3.4e3,
            '1-4': 5e3,
            '2-1': 3.4e3,
            '2-2': 3.4e3,
            '2-3': 5e3,
            '2-4': 3.4e3}

#PCB series resistance dictionary
Rs_pcb = {'1': 5e3,
         '2': 5e3,
         '3': 10e3,
         '4': 10e3,
         '5': 5e3,
         '6': 5e3,
         '7': 10e3,
         '8': 5e3}

#PCB to fridge line mapping
PCB_to_fridge = {'1': '1-1',
                '2': '1-2',
                '3': '1-3',
                '4': '1-4',
                '5':  '2-4',
                '6':  '2-3',
                '7':  '2-2',
                '8':  '2-1'}

In [34]:
line = '7'
Total_Rs =  Rs_pcb[line] + Rs_fridge[PCB_to_fridge[line]]
print(Total_Rs)

13400.0


In [40]:
class Device(InstrumentChannel):
    
    def __init__(self, parent: Instrument, name: str, channel: str, device_info) -> None:
        """
        Args:
            parent: The Instrument instance to which the channel is
                to be attached.
            name: The 'colloquial' name of the channel
            channel: The name used by the Keithley, i.e. either
                'smua' or 'smub'
        """
        super().__init__(parent, name)

        self.add_parameter('device_id',
                               parameter_class=ManualParameter,
                               initial_value=channel,
                               label='Device_ID',
                          )
        
        self.add_parameter('device_type',
                               parameter_class=ManualParameter,
                               initial_value=device_info[0],
                               label='Device_Type',
                               vals=vals.Strings()
                          )
                           
        
        for elec in device_info[1]:
            self.add_parameter(elec,
                                   parameter_class=ManualParameter,
                                   initial_value=0.0,
                                   label=elec,
                                   vals=vals.Numbers()
                              )      

        self.channel = channel

class Die(Instrument):
    def __init__(self, name, fridge, motherboard, daughterboard, device_tuple, **kwargs):
        super().__init__(name, **kwargs)
    
        self.add_parameter('fridge',
                               parameter_class=ManualParameter,
                               initial_value=fridge,
                               label='Fridge',
                               vals=vals.Enum('QT1', 'QT2', 'QT3', 'QT4', 'QT5', 'LC1'))

        self.add_parameter('motherboard',
                               parameter_class=ManualParameter,
                               initial_value=motherboard,
                               label='Motherboard',
                               vals=vals.Enum('Gen4.0', 'Gen4.5', 'Gen5.0'))

        self.add_parameter('daughterboard',
                               parameter_class=ManualParameter,
                               initial_value=daughterboard,
                               label='Daughterboard',
                               vals=vals.Enum('Gen4.0_Rect', 'Gen4.0_Sq', 'Gen4.5', 
                                                 'Gen5.0_Rect_Gnd', 'Gen5.0_Sq_Gnd', 
                                                 'Gen5.0_Rect_Flt', 'Gen5.0_Sq_Flt'))
        
        
        for ch in range(len(device_tuple)):
                ch_name = 'dev{}'.format(ch + 1)
                channel = Device(self, ch_name, ch_name, device_tuple[ch])
                self.add_submodule(ch_name, channel)

In [41]:
electrodes1 = {'source': 1, 'drain': 5, 'cutter1': 2, 'cutter2': 8, 'plunger1': 3, 'plunger2': 7}
# maps PCB pins to device's electrodes
electrodes2 = {'source': 9, 'drain': 10, 'cutter1': 11, 'cutter2': 12, 'plunger1': 13, 'plunger2': 14}
device_list = (('ZBP', electrodes1), ('QD', electrodes2))

chip = Die('V0_0121A', 'QT3', 'Gen5.0', 'Gen5.0_Rect_Flt', device_list)
# name of the die/chip is from the fab

In [42]:
chip.dev1.cutter1()

0.0

In [43]:
chip.dev1.device_type()

'ZBP'

# Trying ideas here

## Concept

In [3]:
class Pin:  # Parameter-like
    """
    This class represents a pin or an electrode.
    
    It has a label that users can refer to when 
    creating connections between pins.
    
    It also has a transformation that defines what 
    happens with the values that are passed thorugh this pin.
    
    TODO:
    * snapshot
    * create properties for useful parts
    * QCODES Parameter-like?
    """
    def __init__(label, transformation):
        self.label = label
        self.transformation = transformation


# probably should be some sort of generic class of 'SmthWithPins'
class Device:  # Instrument-like
    """
    The device represents one device on a chip/die.
    
    It contains pins. They are accessible as attributes.
    
    TODO:
    * adding pins
    * snapshot
    * properties?
    * make loadable/creatable from yaml file
    """
    def __init__(name):
        self.name = name
        # self.add_pin(...)


class Chip:  # Instrument-like
    """
    Just comprises one or more devices, nothing more.
    
    TODO:
    * add device
    * properties?
    * should pins of devices be directly accessible or only via devices attributes?
    * make loadable/creatable from yaml file
    * snapshot
    """
    def __init__(name):
        self.name = name
        # self.add_device(...)
        

Daughterboard = Device
Motherboard = Device
Fridge = Device
BreakoutBox = Device
# TODO: what for the case of MDAC? a wrapper?
# TODO: viasualization of all connections


class Transformation:
    """
    Interface for defining how value is propagated forward and backward
    
    TODO:
    * should it work on value or on Parameter or Pin?
    """
    def forward(self, value):
        raise NotImplemented
    
    def backward(self, value):
        raise NotImplemented


class UnityTransformation(Transformation):
    def forward(self, value):
        return value
    def backward(self, value):
        return value

## Dirty implementation based on qcodes parameters and instruments

In [144]:
from pprint import pprint


In [196]:
from pytopo.pinmap.connect import connect_pins
from pytopo.pinmap.devices import Device, Chip, Daughterboard
from pytopo.pinmap.transformations import AddOneTransformation

Now trying out really:

In [197]:
qubit = Device('qubit')
qubit.add_pin('source', '1')
qubit.add_pin('drain', '2')
qubit.add_pin('cutter', '3')

In [198]:
majorana = Device('majorana')
majorana.add_pin('source', '21')
majorana.add_pin('drain', '22')
majorana.add_pin('plunger', '33')

In [199]:
chip = Chip('chip-1255GHER')
chip.add_device(qubit)
chip.add_device(majorana)

In [200]:
print(chip)

Chip 'chip-1255GHER'
Device 'qubit'
Pin 'source' - '1'
Pin 'drain' - '2'
Pin 'cutter' - '3'
Device 'majorana'
Pin 'source' - '21'
Pin 'drain' - '22'
Pin 'plunger' - '33'


In [201]:
daughter = Daughterboard('daughter-D123')
for i in range(8):
    daughter.add_pin(f'p{i}', f'{i}', transformation=AddOneTransformation)

In [202]:
print(daughter)

Device 'daughter-D123'
Pin 'p0' - '0'
Pin 'p1' - '1'
Pin 'p2' - '2'
Pin 'p3' - '3'
Pin 'p4' - '4'
Pin 'p5' - '5'
Pin 'p6' - '6'
Pin 'p7' - '7'


In [203]:
chip_to_daughter = {
    '1': '1',
    '2': '3',
    '3': '5'
}

In [204]:
connect_pins(of=chip, to=daughter, using_map=chip_to_daughter)

done!


In [205]:
daughter.p5(12)
print(daughter.p5())
print(daughter.p5.raw_value)

12
11


In [206]:
chip.qubit.cutter()

0